#### Preface
В этом практикуме многое позаимствовано у моего коллеги, преподавателя компьютерной лингвистики Миши Нефедова. Спасибо ему.

## Что такое извлечение ключевых слов

Извлечение ключевых слов (keyword extraction) - способ извлечения информации из текста и анализа его тематики. В отличие от автоматического реферирования (саммаризации), мы не создаем полноценный пересказ, а вытаскиваем только ключевые слова (Keywords). Иногда идут чуть дальше и извлекают ключевые словосочетания/n-граммы (Keyphrases). Например, чтобы для текста про "морских коров" не извлекались слова "корова" и "морской", а извлекалось все словосочетание. Это еще актуальнее для более аналитических языков вроде английского (ср. "хотдог" - "hot dog"; вряд ли вы хотите ключевое слово "собака" в тексте про хотдоги).

## Зачем это нужно?

Задача извлечения ключевых слов обычно решается в информационном поиске. Если вы решите писать свой поисковик, или просто индесировать какую-то большую коллекцию текстов, понимать эту тему необходимо. Но и для других задач это может быть полезно. В каком-то смысле, извлечение ключевых слов — простейший способ "тематического моделирования" корпуса. 

## К делу!

### План такой: 
1. Подумать, какие подходы возможны. Реализовать их на случайном примере, оценить результаты глазами

1. Взять готовый набор данных, где ключевые слова уже выделены людьми, и попробовать сравнить с ними (как с эталоном) результат наших методов. Важно помнить, что понятие "эталона" здесь условно. Задача keyword extraction допускает альтернативные решения для одного текста. 

In [ ]:
import os ## работаем с файлами, значит, наверняка понадобится ос
from nltk import word_tokenize ## работаем со словами — значит, токенизатор для этих файлов понадобится
import string ## возьмем оттуда punctuation

## Подготовка

Давайте возьмем для эксперимента 5 текстов: 
* новость РБК про сокращение трат россиян
* новость Медузы про тестирование на коронавирус
* сказка "Колобок"
* повесть Пушкина "Капитанская дочка" 
* рассказ Чехова "Злоумышленник". 

Скачать можно [тут](https://github.com/dhhse/dhcourse/tree/gh-pages/keywords/samples_13_04.zip). У меня они лежат в папке 'samples_13_04' рядом с кодом. Положу этот путь в переменную:

In [ ]:
PATH_TO_TEXTS = 'samples_13_04'

Теперь запишу тексты всех файлов в один список, чтобы потом с ними легче работать: 

In [ ]:
## в этот пустой список я потом запишу тексты файлов, чтобы потом применять к ним разные методы:
file_texts = []
# в этом цикле я сложу в file_texts тексты файлов, лежащие по адресу в PATH_TO_TEXTS:
for some_file in os.listdir (PATH_TO_TEXTS): 
    if some_file.endswith ('.txt'):
        with open (os.path.join(PATH_TO_TEXTS, some_file),'r') as open_file:
            file_texts.append (open_file.read())

## Пробуем подходы

### Самый тупой вариант: берем первые и последние слова текста

Напишем фукнцию, которая берет первые и последние слова (их число задается пользователем как параметр функции).

In [ ]:
def keywords_firstlast (some_text, num_first, num_last):
    """На вход -- строка с текстом some_text, число слов от начала num_first, число слов от конца num_last"""
    ## разобьем текст на токены
    tokenized_text = word_tokenize (some_text.lower())
    if len (tokenized_text) > num_first + num_last:
        if num_last != 0: 
            return tokenized_text [:num_first] + tokenized_text [num_last*-1:]
        else: 
            return tokenized_text [:num_first] # спецобработка случая с нулем слов с конца
    else:
        return tokenized_text

Прогоним по текстам

In [ ]:
for text in file_texts:
    print (keywords_firstlast (text, 6,6))

Получилось не очень. А еще мешают знаки препинания... Давайте их отфильтруем (на этапе заполнения переменной tokenized_text). При добавлении в tokenized_text будем проверять, что помещаемый в список элемент не является пунктуатором). Сначала напишем это в виде цикла с if-ом внутри:

In [ ]:
def keywords_firstlast_no_punct (some_text, num_first, num_last):
    """На вход -- строка с текстом some_text, число слов от начала num_first, число слов от конца num_last"""
    tokenized_text = []
    # пройдемся циклом по результату токенизации текста
    for word in word_tokenize(some_text.lower()):
        # выцепим все слова, которые не входят в string.punctuation
        if word not in string.punctuation:
            # сложим их в переменую tokenized_text, с которой мы работаем дальше
            tokenized_text.append (word)
    if len (tokenized_text) > num_first + num_last:
        if num_last != 0: 
            return tokenized_text [:num_first] + tokenized_text [num_last*-1:]
        else: 
            return tokenized_text [:num_first] # спецобработка случая с нулем слов с конца
    else:
        return tokenized_text

In [ ]:
for text in file_texts:
    print (keywords_firstlast_no_punct (text, 6,6))

А теперь сделаем этот код компактнее, использовав генератор списка, о которых как раз говорил вам Борис Валерьевич на последем занятии.

In [ ]:
def keywords_firstlast_no_punct (some_text, num_first, num_last):
    """На вход -- строка с текстом some_text, число слов от начала num_first, число слов от конца num_last"""
    tokenized_text = [word for word in word_tokenize(some_text.lower()) if word not in string.punctuation]
    if len (tokenized_text) > num_first + num_last:
        if num_last != 0: 
            return tokenized_text [:num_first] + tokenized_text [num_last*-1:]
        else: 
            return tokenized_text [:num_first] # спецобработка случая с нулем слов с конца
    else:
        return tokenized_text

Посмотрим еще раз:

In [ ]:
for text in file_texts:
    print (keywords_firstlast_no_punct (text, 6,6))

Уже лучше, но в стандартную сборку string.punctuation явно входит не все, что нам нужно. Усовершенствуем ее:

In [ ]:
extended_punctuation = string.punctuation + '—»«...' 

Поменяем функцию, добавив туда extended_punctuation вместо string.punctuation

In [ ]:
def keywords_firstlast_no_punct (some_text, num_first, num_last):
    """На вход -- строка с текстом some_text, число слов от начала num_first, число слов от конца num_last"""
    tokenized_text = [word for word in word_tokenize(some_text.lower()) if word not in extended_punctuation]
    if len (tokenized_text) > num_first + num_last:
        if num_last != 0: 
            return tokenized_text [:num_first] + tokenized_text [num_last*-1:]
        else: 
            return tokenized_text [:num_first] # спецобработка случая с нулем слов с конца
    else:
        return tokenized_text

In [ ]:
for text in file_texts:
    print (keywords_firstlast_no_punct (text, 6,6))

### Менее тупой вариант: берем самые частотные слова текста

In [ ]:
from nltk import FreqDist # как вы помните, в нлтк есть счетчик частотностей
# но можно и пользоваться Counter из модуля collections

Краткое напоминание: как работает FreqDist:

In [ ]:
text = 'повар петр повар павел петр пек'
FreqDist(text.split())

In [ ]:
FreqDist(text.split()).most_common (2)

In [ ]:
def keywords_most_frequent (some_text, num_most_freq):
    """На вход -- строка с текстом some_text, число самых частотных слов от начала num_most_freq"""
    # запишем в переменную tokenized_text список токенов без пунктуации
    tokenized_text = [word for word in word_tokenize (some_text.lower()) if word not in extended_punctuation] 
    # вернем самые частотные слова, посчитанные с помощью метода most_common у FreqDist 
    return [word_freq_pair[0] for word_freq_pair in FreqDist(tokenized_text).most_common(num_most_freq)]

In [ ]:
for text in file_texts:
    print (keywords_most_frequent (text, 10))

### Третий вариант: берем самые частотные слова текста без стоп-слов

In [ ]:
with open ('stop_ru.txt', 'r') as stop_file:
    rus_stops = [word.strip() for word in stop_file.readlines()] # запишем стослова в список 

In [ ]:
def keywords_most_frequent_with_stop (some_text, num_most_freq, some_stoplist):
    tokenized_text = [word for word in word_tokenize (some_text.lower()) if word not in extended_punctuation and word not in some_stoplist]
    return [word_freq_pair[0] for word_freq_pair in FreqDist(tokenized_text).most_common(num_most_freq)]


In [ ]:
for text in file_texts:
    print (keywords_most_frequent_with_stop (text, 6, rus_stops))

### Четвертый вариант: берем самые частотные ЛЕММЫ текста без стоп-слов

In [ ]:
from pymystem3 import Mystem ## используем mystem в обертке pymystem

In [ ]:
moi_analizator = Mystem() ## создаем экземпляр класса "анализатор MyStem"  

In [ ]:
def keywords_most_frequent_with_stop_and_lemm (some_text, num_most_freq, some_stoplist):
    lemmatized_text = [word for word in moi_analizator.lemmatize(some_text.lower()) if word.strip() not in extended_punctuation and word not in some_stoplist]
    return [word_freq_pair[0] for word_freq_pair in FreqDist(lemmatized_text).most_common(num_most_freq)]


In [ ]:
for text in file_texts:
    print (keywords_most_frequent_with_stop_and_lemm (text, 10, rus_stops))

Майстем наловил нам всякой пунктуации. Нужны еще фильтры. А вторая строчка функции разрослась и не читается. Давайте вынесем все эти фильтры в отдельную функцию:

In [ ]:
def passed_filter (some_word, stoplist):
    some_word = some_word.strip()
    if some_word in extended_punctuation:
        return False
    elif some_word in stoplist:
        return False
    elif re.search ('[А-ЯЁа-яёA-Za-z]', some_word) == None:
        return False
    return True

In [ ]:
def keywords_most_frequent_with_stop_and_lemm (some_text, num_most_freq, stoplist):
    lemmatized_text = [word for word in moi_analizator.lemmatize(some_text.lower()) if passed_filter(word, stoplist)]
    return [word_freq_pair[0] for word_freq_pair in FreqDist(lemmatized_text).most_common(num_most_freq)]

In [ ]:
for text in file_texts:
    print (keywords_most_frequent_with_stop_and_lemm (text, 10, rus_stops))

## А какие-нибудь более продвинутые варианты будут?

## 1. Meet TF-IDF ! 

TF IDF — это мера, которая учитывает не только частотность слова в документе (TF, term frequency) — но и то, насколько часто — вернее, насколько редко! — оно встречается во всем корпусе (IDF, inverse document frequency). Формально она считается так:

<img src="pics/tfidf.gif">

Суть в том, что таким образом повышаются слова, частотные в данном документе -- и редкие в корпусе в целом. Такие слова получают бонус относительно слов, частотных и в данном документе, и в других. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer ## готовая реализация TF IDF из библиотеки sklearn

In [ ]:
make_tf_idf = TfidfVectorizer (stop_words=rus_stops) # Создаем специальный объект-векторайзер 
#fitted_vectorizer = make_tf_idf.fit(file_texts) 
texts_as_tfidf_vectors=make_tf_idf.fit_transform(file_texts) # Кладем в этот векторайзер наши файлы и просим сделать матрицу TF_IDF



In [ ]:
texts_as_tfidf_vectors

In [ ]:
print (texts_as_tfidf_vectors.shape) ## посмотрим размерность матрицы

In [ ]:
make_tf_idf.get_feature_names()[100:200] ## посмотрим 

In [ ]:
## кусочек кода, который берет матрицу TF-IDF и выдает по ней топ-слова для каждого текста

## в словарь id2word запишем соответствия между числовами индексами слов, которые хранятся 
## в матрице tfidf -- и самими словами:
id2word = {i:word for i,word in enumerate(make_tf_idf.get_feature_names())} 

# теперь пройдемся по матрице и вытащим для каждого текста слова с самым большим tfidf

for text_row in range(texts_as_tfidf_vectors.shape[0]):
    ## берем ряд в нашей матрице -- он соответстует тексту
    row_data = texts_as_tfidf_vectors.getrow(text_row) 
    ## сортируем в нем все слова (вернее, индексы слов) -- получаем от самых маленьких к самым большим
    words_for_this_text = row_data.toarray().argsort() 
    ## берем крайние 6 слов отсортированного ряда
    top_words_for_this_text = words_for_this_text [0,:-6:-1] 
    print ([id2word[w] for w in top_words_for_this_text]) ## 

### Естественно, тут можно снова накрутить лемматизацию 

пусть текст лемматизируется в отдельной функции 

In [ ]:
def preprocess_for_tfidif (some_text):
    lemmatized_text = moi_analizator.lemmatize(some_text.lower())
    return (' '.join(lemmatized_text)) # поскольку tfidf векторайзер принимает на вход строку, 
    #после лемматизации склеим все обратно

Сам код для расчета tfidf на корпусе текстов давайте тоже упакуем в функцию:

In [ ]:
def produce_tf_idf_keywords (some_texts, number_of_words):
    make_tf_idf = TfidfVectorizer (stop_words=rus_stops)
    texts_as_tfidf_vectors=make_tf_idf.fit_transform(preprocess_for_tfidif(text) for text in some_texts)
    id2word = {i:word for i,word in enumerate(make_tf_idf.get_feature_names())} 

    for text_row in range(texts_as_tfidf_vectors.shape[0]): 
        ## берем ряд в нашей матрице -- он соответстует тексту:
        row_data = texts_as_tfidf_vectors.getrow(text_row)
        ## сортируем в нем все слова: 
        words_for_this_text = row_data.toarray().argsort() 
        ## берем число слов с конца, равное number_of_words 
        top_words_for_this_text = words_for_this_text [0, :-1*(number_of_words+1):-1]
        ## печатаем результат
        print([id2word[w] for w in top_words_for_this_text])


In [ ]:
produce_tf_idf_keywords (file_texts, 6)

## 2. Еще один умный способ: превратить текст в граф (сеть) и искать центральные узлы

Много методов для извлечения ключевых слов основаны на сетевом анализе. Основная идея - каким-то образом перевести текст в граф, а затем посчитать центральности узлов и вывести центральные.  

<img src="https://www.researchgate.net/profile/Mitsuru_Ishizuka/publication/2539694/figure/fig1/AS:669437975883786@1536617859438/A-word-cooccurrence-graph-of-a-set-of-news-articles-The-source-articles-are-a-set-of.png">

Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет основой графа. Ниже приведена НЕ моя реализация сборки такого графа по тексту и примения к нему алгоритма random walk для выделения ключевых слов. Вот описание от автора:

Для выбора важных узлов часто используют простой random walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [ ]:
from itertools import combinations
from collections import Counter
import np

In [ ]:
### не моя реализация алгоритма

def get_kws(text, top=6, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):

        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        
        ### 
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n

    

Испытаем эту реализацию на наших текстах

In [ ]:
for text in file_texts:
    print (get_kws (word_tokenize(text))) # функция принимает на вход список слов, а не строку, поэтому так

Ах да, нам ведь опять надо делать предобработку типа удаления пунктуации и стоп-слов. Давайте уже сделаем общую функцию для этого:

In [ ]:
def preprocessing_general (input_text, stoplist):
    '''функция для предобработки текста; 
    на вход принимает строку с текстом input_text и список стоп-слов stoplist
    на выходе чистый список слов output'''
    ## лемматизируем майстемом и делаем strip каждого слова:
    output = [word.strip() for word in moi_analizator.lemmatize (input_text)] 
    ## убираем пунктуацию и стоп-слова:
    output = [word for word in output if word not in extended_punctuation and word not in stoplist]
    ## убираем слова, в которых вообще нет буквенных символов:
    output = [word for word in output if re.search ('[А-ЯЁа-яёA-Za-z]', word) != None]
    return output

In [ ]:
for text in file_texts:
    print (get_kws (preprocessing_general(text, rus_stops)))

Замечание: в отличие от всех прочих, этот метод недетерминированный. Поскольку в функции происходит случайное хождение по узлам, даже без смены параметров результаты будут отличаться от прогона к прогону

## Вторая часть плана: тестируем на реальных данных

1. Возьмем готовые данные, в которых уже размечены ключевые слова. 

2. Напишем код для сравнения этих keywords с теми, что выделяют наши функции.

### 1. Берем реальные данные

Данные возьмем отсюда: https://github.com/mannefedov/ru_kw_eval_datasets Тут лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). В них уже размечены keywords

Вы можете скачать их все при помощи `git clone https://github.com/mannefedov/ru_kw_eval_datasets`

### 2. Считываем данные 

Я буду работать с одним файлом — ng_1.jsonlines. Как можно догадаться по названию, это файл в формате json (вы уже [познакомились с этим форматом](https://agricolamz.github.io/DS_for_DH/lists.html) в курсе Гарика). Jsonlines -- версия формата JSON, в которой хранится много json-объектов с новой строки. Поэтому целый файл не получится прочитать целиком с помощью стандартного питоновского json.load. Давайте пробовать:

In [ ]:
import json

In [ ]:
with open("data/ng_1.jsonlines", "r") as read_file:
    ng_1_data = json.load(read_file)

Выдает ошибку парсинга json. Зато можно вот так: 

In [ ]:
ng_1_data = []
with open("data/ng_1.jsonlines", "r") as read_file:
    for line in read_file:
        ng_1_data.append(json.loads(line)) # json.loads считывает строку, в отличие от json.load

Теперь в списке "ng_1_data" лежат объекты из нашего json. Всего 988:

In [ ]:
len (ng_1_data)

Каждый элемент списка соответствует тексту, у которого есть кроме самого текста заголовок, набор приписанных вручную ключевых слов, URL и краткий пересказ (summary)

In [ ]:
ng_1_data[0]

С технической точки зрения при таком считывании JSON в питон (по-умному это называется "десериализация") JSON-объекты превращаются в словари. По ключам можно доставать значения:

In [ ]:
ng_1_data[0] ['title']

In [ ]:
for item in ng_1_data[:4]:
    print (item['keywords'])

В принципе стандартнымх ходом здесь было бы положить весь json внутрь pandas.DataFrame и работать с датафреймом... Но если считать, что pandas мы с вами еще не проходили, то можно обойтись и без него. 

М.б. это будет чуть менее красиво -- зато все собрано из самых простых подручных материалов (циклы + списки).

### 3. Итак, наконец-то применяем наши наработки по извлечению ключевых слов

С самыми примитивными решениями, которые мы придумали, можно вообще за один цикл все посмотреть: 

In [ ]:
for item in ng_1_data[:10]:
    print ('Эталонные ключевые слова: ', item['keywords'])
    print ('Первые и последние слова', keywords_firstlast (item['content'], 4,4))
    print ()

Что-то не особо сходится... Попробуем частотные:

In [ ]:
for item in ng_1_data[:10]:
    print ('Эталонные ключевые слова: ', item['keywords'])
    print ('Самые частотные слова: ',  keywords_most_frequent_with_stop_and_lemm (item['content'], 6, rus_stops))
    print ()

Кажется, что совпадения есть... 

Но с TF-IDF мы аналогично сделать не можем -- ведь ее расчет для одного текста требует знания о всех текстах (и поэтому наша функция produce_tf_idf_keywords принимает на вход список текстов, а не одну текстовую строку).

Поэтому мы сначалапройдемся по считанным из json данным и заполним списки

In [ ]:
manual_keywords = [] ## сюда запишем все ключевые слова, приписанные вручную
full_texts = [] ## сюда тексты

In [ ]:
for item in ng_1_data:
    manual_keywords.append(item['keywords'])
    full_texts.append(item['content'])

Теперь можно применять к списку с текстами нашу старую функцию tf-idf. Давайте применим ее сначала к небольшому подмножеству из пары десятков текстов:
    

In [ ]:
produce_tf_idf_keywords (full_texts[:20], 6) 

In [ ]:
manual_keywords [:20]

Выглядит уже неплохо -- а ведь может стать и лучше, если использвать весь корпус, а не только 20 текстов.

Вопрос в том, как нам оценить это "неплохо". 

## Оценка качества извлечения ключевых слов 
### или немного про точность и полноту 

Наверно, нам надо как-то считать процент попаданий. Например, сколько слов из эталона накрыл наш алгоритм. 

Для начала создадим список предсказанных нами ключевых слов. Для этого придется немножко видоизменить функцию для tfidf -- ведь она раньше просто печатала  keywords для каждого текста -- а нам надо, чтобы она вернула последовательный список keywords для всех текстов.

In [ ]:
def produce_tf_idf_keywords (some_texts, number_of_words):
    result = []
    make_tf_idf = TfidfVectorizer (stop_words=rus_stops)
    texts_as_tfidf_vectors=make_tf_idf.fit_transform(preprocess_for_tfidif(text) for text in some_texts)
    id2word = {i:word for i,word in enumerate(make_tf_idf.get_feature_names())} 
    for text_row in range(texts_as_tfidf_vectors.shape[0]): 
        row_data = texts_as_tfidf_vectors.getrow(text_row) ## берем ряд в нашей матрице -- он соответстует тексту
        words_for_this_text = row_data.toarray().argsort() ## сортируем в нем все слова 
        top_words_for_this_text = words_for_this_text [0,:-1*number_of_words:-1] 
        result.append([id2word[w] for w in top_words_for_this_text])
    return (result)

In [ ]:
predicted_keywords = produce_tf_idf_keywords (full_texts, 6) 

In [ ]:
predicted_keywords [7]

In [ ]:
manual_keywords [7]

In [ ]:
for index in range(20):
    print (manual_keywords [index])
    print (predicted_keywords[index])

In [ ]:
def simple_match_counter (list_a, list_b):
    '''считает среднее всех пересечений (доли угаданных нами ключ.слов для каждого текста)'''
    all_matches = []
    for index, words_a in enumerate (list_a):
        words_b = list_b [index]
        intersection = len (set(words_a) & set (words_b)) #  число элементов в пересечении списков
        all_matches.append (intersection/len(words_a))
    return sum(all_matches)/ len (all_matches)

In [ ]:
simple_match_counter (manual_keywords, predicted_keywords)

Сравнимся с простой частотностью слов:

In [ ]:
predicted_keywords_firstlast = [keywords_firstlast (text, 6, 6) for text in full_texts]

In [ ]:
simple_match_counter (manual_keywords, predicted_keywords_firstlast)

In [ ]:
predicted_keywords_firstlast_no_punct = [keywords_firstlast_no_punct (text, 6, 6) for text in full_texts] 

In [ ]:
simple_match_counter (manual_keywords, predicted_keywords_firstlast_no_punct)

In [ ]:
predicted_keywords_freq_lemm = [keywords_most_frequent_with_stop_and_lemm(text, 6, rus_stops) for text in full_texts]

In [ ]:
simple_match_counter (manual_keywords, predicted_keywords_freq_lemm)

### Проблема

Проблема такого подхода — легко повысить результат, просто увеличив число выдаваемых слов для одного текста. Ведь тогда мы наверняка накроем больше верных

In [ ]:
predicted_keywords_10_per_text = produce_tf_idf_keywords (full_texts, 10) 

In [ ]:
simple_match_counter (manual_keywords, predicted_keywords_10_per_text)

Можно поменять местами списки на входе функции — и тогда мы будем искать не сколько слов в эталоне мы накрыли, а сколько из тех слов, что мы выдали, есть в эталоне

In [ ]:
simple_match_counter (predicted_keywords_10_per_text, manual_keywords)

Но тут появляется возможность читерства в обратную сторону: выдавать как можно меньше слов для одного текста

In [ ]:
predicted_keywords_2_per_text = produce_tf_idf_keywords (full_texts, 2) 

In [ ]:
simple_match_counter (predicted_keywords_2_per_text, manual_keywords)

### Точность, полнота и F-мера

#### Полнота
Когда мы вначале смотрели, какая часть слов в эталоне накрыта нашей predicted-выдачей, мы, в сущности, считали __полноту__ (recall) нашего алгоритма. То есть какую долю всех верных ответов мы находим (не принимая во внимание свои "неверные"). 

#### Точность
Когда мы поменяли местами predicted и manual, функция стала считать, какая часть слов в выдаче есть в эталоне, мы, в сущности, считали __точность__ (precision) нашего алгоритма. То есть какую долю среди наших ответов составляют верные ответы (не принимая во внимание верные ответы эталона, которые мы не выдали). 

#### F-мера

Я попытался показать, что и ту, и другую метрику можно хакнуть — они слишком однобоки. Ученые это давно поняли и придумали усредняющую метрику — F1-меру. Это "гармоническое среднее" точности и полноты:

<img src = "pics/fmeasure.png">

Немножко доработаем нашу функцию simple_match_counter , чтобы она считала сразу точность, полноту и F-меру

In [ ]:
def precision_recall_fmeasure (manual, predicted):
    '''считает точность, полноту и F-меру'''
    precisions = []
    recalls = []

    for index, words_manual in enumerate (manual):
        words_predicted = predicted [index]
        intersection = len (set(words_manual) & set (words_predicted)) #  число элементов в пересечении списков
        recalls.append (intersection/len(words_manual)) 
        precisions.append (intersection/len(words_predicted))
        
    mean_precision = sum(precisions)/ len (precisions)
    mean_recall = sum(recalls)/ len (recalls)
    fmeasure =  ((2*mean_recall*mean_precision)/(mean_recall+mean_precision))
    return 'Точность: {}, полнота: {}, F-мера {}'.format (mean_precision, mean_recall, fmeasure)

In [ ]:
precision_recall_fmeasure (manual_keywords, predicted_keywords_freq_lemm)

## Задание по ключевым словам: 

1. Напишите сами любой алгоритм извлечения ключевых слов. Какой угодно, пусть даже интуитивно он будет нелепый, неважно. Конечно, будет интересно придумать что-то хитрое, с опорой на лингвистику (пока ее тут почти что не было). Или свой графовый алгоритм -- можно с networkx, а не на матрицах:) Еще круче, если вы сделаете не только слова, но и фразы (чтобы был шанс выловить "образовательные стандарты" или "леонида юзефовича" целиком). Но можно и воспроизвести что-то из того, что есть выше. Но только не копируйте код, а пишите сами, воспроизводя логику. 

2. Возьмите любой собственный набор текстов. Примените алгоритм к текстам, выведите результаты на экран.

2. Протестируйте алгоритм на любом документе (документах) из датасета https://github.com/mannefedov/ru_kw_eval_datasets Будет хорошо, если вы напишете свою функцию измерения точности, полноты и F-меры. Желающие могут добавить туде еще расчет [кэффициента близости Жаккара](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D1%8D%D1%84%D1%84%D0%B8%D1%86%D0%B8%D0%B5%D0%BD%D1%82_%D0%96%D0%B0%D0%BA%D0%BA%D0%B0%D1%80%D0%B0). 

3. Сравните результат с тем, который получается, если просто брать все частотные леммы.

4. Результат в любом виде (.py, .ipynb) выкладывайте на свой гитхаб.  